In [1]:
'''
Author: Bryan Bo Cao
Email: boca7588@colorado.edu or bo.cao-1@colorado.edu
Github Repo: https://github.com/BryanBo-Cao/neuralnets-deeplearning
'''
from __future__ import print_function

import tensorflow as tf
import numpy as np
import random
import copy
import matplotlib.pyplot as plt

# Reading data
all_output_0_train_accuracy = 0.787670391748
all_output_0_test_accuracy = 0.789889253486
all_output_1_train_accuracy = 0.212329608252
all_output_1_test_accuracy = 0.210110746514

train_data = np.genfromtxt('train_data.txt', delimiter=',')
#print "train_data:", train_data

t = train_data[:,2]
hu = train_data[:,3]
lt = train_data[:,4]
co2 = train_data[:,5]
hu_r = train_data[:,6]

o = train_data[:,7]

#data
data = np.column_stack((t, hu, lt, co2, hu_r))
data = np.float32(data)
#print ("data:", data)

#print "np.shape(train_data): ", np.shape(train_data)
#print "np.shape(t): ", np.shape(t)
'''
print ("t: ", t)
print ("hu: ", hu)
print ("lt: ", lt)
print ("co2: ", co2)
print ("hu_r: ", hu_r)
print ("o: ", o)
'''
x_data = np.array(data)
y_data = np.array(o)
print ([x_data, y_data])

[array([[  2.31800003e+01,   2.72719994e+01,   4.26000000e+02,
          7.21250000e+02,   4.79298830e-03],
       [  2.31499996e+01,   2.72674999e+01,   4.29500000e+02,
          7.14000000e+02,   4.78344085e-03],
       [  2.31499996e+01,   2.72450008e+01,   4.26000000e+02,
          7.13500000e+02,   4.77946363e-03],
       ..., 
       [  2.11000004e+01,   3.60950012e+01,   4.33000000e+02,
          7.98500000e+02,   5.59563888e-03],
       [  2.11000004e+01,   3.62599983e+01,   4.33000000e+02,
          8.20333313e+02,   5.62144956e-03],
       [  2.11000004e+01,   3.62000008e+01,   4.47000000e+02,
          8.21000000e+02,   5.61206369e-03]], dtype=float32), array([ 1.,  1.,  1., ...,  1.,  1.,  1.])]


In [2]:
def normalize(data):
    #normalize train data
    #normalize t
    t = data[:, 0]
    t_min = np.min(t)
    t_max = np.max(t)
    d_t = t_max - t_min
    #normalized t: n_t
    n_t = []
    for each in t:
        n_t.append((each - t_min) / d_t)

    #normalize hu
    hu = data[:, 1]
    hu_min = np.min(hu)
    hu_max = np.max(hu)
    d_hu = hu_max - hu_min
    #normalized h: n_hu
    n_hu = []
    for each in hu:
        n_hu.append((each - hu_min) / d_hu)

    #normalize lt
    lt = data[:, 2]
    lt_min = np.min(lt)
    lt_max = np.max(lt)
    d_lt = lt_max - lt_min
    #normalized lt: n_lt
    n_lt = []
    for each in lt:
        n_lt.append((each - lt_min) / d_lt)

    #normalize co2
    co2 = data[:, 3]
    co2_min = np.min(co2)
    co2_max = np.max(co2)
    d_co2 = co2_max - co2_min
    #normalized co2: n_co2
    n_co2 = []
    for each in co2:
        n_co2.append((each - co2_min) / d_co2)

    #normalize hu_r
    hu_r = data[:, 4]
    hu_r_min = np.min(hu_r)
    hu_r_max = np.max(hu_r)
    d_hu_r = hu_r_max - hu_r_min
    #normalized hu_r: n_hu_r
    n_hu_r = []
    for each in hu_r:
        n_hu_r.append((each - hu_r_min) / d_hu_r)

    #normalized data: n_data
    n_data = np.column_stack((n_t, n_hu, n_lt, n_co2, n_hu_r))

    return n_data

n_data = normalize(data)
print ("n_data:", n_data)
x_data = np.array(n_data)
y_data = np.array(o)
print ([x_data, y_data])

n_data: [[ 1.          0.47053295  0.2754904   0.190933    0.55731839]
 [ 0.99282283  0.47033185  0.27775383  0.18644592  0.55480719]
 [ 0.99282283  0.4693262   0.2754904   0.18613647  0.55376107]
 ..., 
 [ 0.50239241  0.86490113  0.28001723  0.23874362  0.76843745]
 [ 0.50239241  0.87227613  0.28001723  0.25225642  0.77522635]
 [ 0.50239241  0.8695944   0.2890709   0.25266904  0.77275759]]
[array([[ 1.        ,  0.47053295,  0.2754904 ,  0.190933  ,  0.55731839],
       [ 0.99282283,  0.47033185,  0.27775383,  0.18644592,  0.55480719],
       [ 0.99282283,  0.4693262 ,  0.2754904 ,  0.18613647,  0.55376107],
       ..., 
       [ 0.50239241,  0.86490113,  0.28001723,  0.23874362,  0.76843745],
       [ 0.50239241,  0.87227613,  0.28001723,  0.25225642,  0.77522635],
       [ 0.50239241,  0.8695944 ,  0.2890709 ,  0.25266904,  0.77275759]], dtype=float32), array([ 1.,  1.,  1., ...,  1.,  1.,  1.])]


In [3]:
n_input = 5
H0 = 5
n_output = 1
output_num_nodes = 1
lr = 0.04
epochs = 4000
bs = 100
data_size = len(x_data)

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

ws0 = tf.Variable(tf.random_uniform([n_input, H0], 0.0, 1.0))
ws1 = tf.Variable(tf.random_uniform([H0, n_output], 0.0, 1.0))

#H0_outputs = tf.sigmoid(tf.matmul(X, ws0))
H0_outputs = tf.matmul(X, ws0)
#hy = tf.sigmoid(tf.matmul(H0_outputs, ws1))
hy = tf.matmul(H0_outputs, ws1)

#cost = tf.reduce_mean(tf.square(Y - hy))
cost = tf.reduce_mean(tf.square(Y - hy) / 2)
#cost = tf.reduce_mean(tf.nn.l2_loss(Y - hy))
'''
x_data_batch, y_data_batch = tf.train.batch(
    [x_data, y_data],
    batch_size = bs)
'''
x_data_batch = tf.train.batch([x_data], batch_size = bs)
y_data_batch = tf.train.batch([y_data], batch_size = bs)

optimizer = tf.train.GradientDescentOptimizer(lr).minimize(cost)

init = tf.global_variables_initializer()

print ("x_data_batch: ", x_data_batch)
print ("y_data_batch: ", y_data_batch)
print ("x_data_batch[0]: ", x_data_batch[0])
print ("tf.cast(x_data_batch[0][0][0], 'float'): ", tf.cast(x_data_batch[0][0][0], "float"))

x_data_batch:  Tensor("batch:0", shape=(100, 8143, 5), dtype=float32)
y_data_batch:  Tensor("batch_1:0", shape=(100, 8143), dtype=float64)
x_data_batch[0]:  Tensor("strided_slice:0", shape=(8143, 5), dtype=float32)
tf.cast(x_data_batch[0][0][0], 'float'):  Tensor("strided_slice_3:0", shape=(), dtype=float32)


In [ ]:
#print (x_data)
print (x_data.shape)
xx_data = x_data[0:100]
print (xx_data.shape)

(8143, 5)
(100, 5)


In [1]:
with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
   
    for step in xrange(epochs):
        
        #train network in batch size = bs
        bs_i = 0
        while bs_i <= data_size:
            batch_end = 0
            
            if bs_i + 100 < data_size:
                batch_end = bs_i + 100
            else:
                batch_end = data_size
            xx_data_batch = x_data[bs_i : batch_end]
            yy_data_batch = y_data[bs_i : batch_end]
            
            sess.run(optimizer, feed_dict={X: xx_data_batch, Y: yy_data_batch})
            bs_i += 100
        
        #sess.run(optimizer, feed_dict={X: xx_data_batch, Y: yy_data_batch})
        
        if step % 10 == 0:
            print ("   ")
            print ("Step:", step)
            #print ("ws1: ", ws1.eval())
            print ("Cost:", sess.run(cost, feed_dict={X: x_data, Y: y_data}))
            #print ("tf.floor(hy + 0.5)", tf.floor(hy + 0.5))
            print ("hy: ", sess.run(hy, feed_dict={X: x_data, Y: y_data}))
            print ("H0_outputs: ", sess.run(H0_outputs, feed_dict={X: x_data, Y: y_data}))
            correct_prediction = tf.equal(tf.floor(hy + 0.5), Y)
            correct_prediction = tf.equal(tf.floor(tf.sigmoid(hy) + 0.5), Y)
            #correct_prediction = tf.equal(hy, Y)
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            #print ("correct_prediction: ", sess.run(correct_prediction, feed_dict={X: x_data, Y: y_data}))
            print ("Accuracy: ", accuracy.eval({X: x_data, Y: y_data}))
            
    correct_prediction = tf.equal(tf.floor(hy + 0.5), Y)
    
    print ("correct_prediction: ", sess.run(correct_prediction, feed_dict={X: x_data, Y: y_data}))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
    print (sess.run([hy], feed_dict={X: x_data, Y: y_data}))
    print ("Accuracy: ", accuracy.eval({X: x_data, Y: y_data}))
    coord.request_stop()
    coord.join(threads)

NameError: name 'tf' is not defined